# Week 3/4: Full Data Collection, Cleaning, and Integration

**Week 3/4: Full Data Collection**
- Execute large-scale API data collection for top 500 cities with error handling
- Validate SimpleMaps dataset for clean identifiers and reliable population data
- Store raw and processed files with consistent naming conventions
- Record timestamps, logs, and collection notes
- Standardize field names, formats, and units
- Merge population, air quality, and weather data
- Run validation checks for duplicates, missing values, inconsistent joins
- Document cleaning and integration steps

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
from config import API_KEY, BASE_URL
import time
from datetime import datetime
import os

# Create log list to track all operations (claude assisted)
curation_log = []

def log_step(step_name, details):
    """Log a curation step with timestamp"""
    timestamp = datetime.now().isoformat()
    log_entry = {
        'timestamp': timestamp,
        'step': step_name,
        'details': details
    }
    curation_log.append(log_entry)
    print(f"[{timestamp}] {step_name}: {details}")

## Part 1: SimpleMaps Dataset Validation

Load and validate the SimpleMaps dataset for data quality.

In [4]:
# Load the full SimpleMaps dataset
cities_df = pd.read_csv('data/worldcities.csv')
log_step('Data Load', f'Loaded {len(cities_df)} cities from SimpleMaps dataset')
print(f"Shape: {cities_df.shape}")
print(cities_df.head(3))

[2025-11-20T14:26:27.387966] Data Load: Loaded 48059 cities from SimpleMaps dataset
Shape: (48059, 11)
      city city_ascii     lat       lng    country iso2 iso3 admin_name  \
0    Tokyo      Tokyo  35.687  139.7495      Japan   JP  JPN      Tōkyō   
1  Jakarta    Jakarta  -6.175  106.8275  Indonesia   ID  IDN    Jakarta   
2    Delhi      Delhi  28.610   77.2300      India   IN  IND      Delhi   

   capital  population          id  
0  primary  37785000.0  1392685764  
1  primary  33756000.0  1360771077  
2    admin  32226000.0  1356872604  


In [ ]:
# Check missing
missing_data = cities_df.isnull().sum()
print(missing_data[missing_data > 0])
missing_summary = missing_data[missing_data > 0].to_dict()

# log
log_step('Validation - Missing Values', f'{len(missing_summary)} columns have missing values')

city_ascii        2
iso2             33
admin_name      201
capital       32921
population      251
dtype: int64
[2025-11-20T14:26:46.632955] Validation - Missing Values: 5 columns have missing values


In [6]:
# duplicates
# log
duplicate_cities = cities_df.duplicated(subset=['city', 'country'], keep=False)
num_duplicates = duplicate_cities.sum()

print(f"Duplicates: {num_duplicates}")
if num_duplicates > 0:
    print(cities_df[duplicate_cities][['city', 'country', 'lat', 'lng']].head(10))

# log
log_step('Validation - Duplicates', f'Found {num_duplicates} duplicate city-country combinations')

Duplicates: 3694
           city        country      lat       lng
37     Dongguan          China  23.0210  113.7520
56       Jining          China  35.4151  116.5871
63       Fuyang          China  32.8900  115.8140
77     Shaoyang          China  27.2395  111.4679
86        Miami  United States  25.7840  -80.2101
94       Dallas  United States  32.7935  -96.7667
116      Suzhou          China  33.6480  116.9640
120   Xiangyang          China  32.0100  112.1220
123  Washington  United States  38.9047  -77.0163
128      Yichun          China  27.8160  114.4170
[2025-11-20T14:28:21.702501] Validation - Duplicates: Found 3694 duplicate city-country combinations


In [7]:
# Population data check
cities_with_pop = cities_df[cities_df['population'].notna()]
pop_coverage = len(cities_with_pop) / len(cities_df) * 100

print(f"Population coverage: {pop_coverage:.1f}%")
print(f"Missing: {cities_df['population'].isna().sum()}")
print(cities_with_pop['population'].describe())

# log
log_step('Validation - Population Data', f'{pop_coverage:.1f}% of cities have population data')

Population coverage: 99.5%
Missing: 251
count    4.780800e+04
mean     1.078566e+05
std      6.855111e+05
min      0.000000e+00
25%      1.219100e+04
50%      2.091350e+04
75%      4.680850e+04
max      3.778500e+07
Name: population, dtype: float64
[2025-11-20T14:28:32.216926] Validation - Population Data: 99.5% of cities have population data


In [ ]:
# Check coordinate validity
invalid_coords = cities_df[
    (cities_df['lat'].isna()) | 
    (cities_df['lng'].isna()) |
    (cities_df['lat'] < -90) | 
    (cities_df['lat'] > 90) |
    (cities_df['lng'] < -180) | 
    (cities_df['lng'] > 180)
]

print(f"Invalid coords: {len(invalid_coords)}")
if len(invalid_coords) > 0:
    print(invalid_coords[['city', 'country', 'lat', 'lng']].head())

# log
log_step('Validation - Coordinates', f'{len(invalid_coords)} cities with invalid coordinates')

Invalid coords: 0
[2025-11-20T14:28:54.540540] Validation - Coordinates: 0 cities with invalid coordinates


In [9]:
# Select top 500 cities by population
valid_cities = cities_df[
    (cities_df['population'].notna()) &
    (cities_df['lat'].notna()) &
    (cities_df['lng'].notna())
].copy()

top_500_cities = valid_cities.nlargest(500, 'population').reset_index(drop=True)

print(f"Pop range: {top_500_cities['population'].min():,.0f} to {top_500_cities['population'].max():,.0f}")
print(top_500_cities[['city', 'country', 'population']].head(10))

# Save validated top 500 cities
top_500_cities.to_csv('data/raw_top_500_cities.csv', index=False)

# log
log_step('Data Selection', 'Selected and saved top 500 cities by population')

Pop range: 1,543,000 to 37,785,000
          city       country  population
0        Tokyo         Japan  37785000.0
1      Jakarta     Indonesia  33756000.0
2        Delhi         India  32226000.0
3    Guangzhou         China  26940000.0
4       Mumbai         India  24973000.0
5       Manila   Philippines  24922000.0
6     Shanghai         China  24073000.0
7    São Paulo        Brazil  23086000.0
8        Seoul  Korea, South  23016000.0
9  Mexico City        Mexico  21804000.0
[2025-11-20T14:29:13.695526] Data Selection: Selected and saved top 500 cities by population


## Part 2: Large-Scale API Data Collection

Collect air quality data for all 500 cities with proper error handling and rate limiting.

In [10]:
def get_current_air_quality(lat, lon, api_key, retry_count=3):
    """Get current air quality with retry logic and error handling"""
    url = f"{BASE_URL}currentConditions:lookup"
    params = {"key": api_key}
    data = {
        "location": {
            "latitude": lat,
            "longitude": lon
        }
    }
    
    for attempt in range(retry_count):
        try:
            response = requests.post(url, params=params, json=data, timeout=10)
            
            if response.status_code == 200:
                return {'status': 'success', 'data': response.json()}
            elif response.status_code == 429:
                # Rate limit hit, wait longer
                time.sleep(2 ** attempt)  # Exponential backoff
                continue
            else:
                return {'status': 'error', 'error_type': 'http_error', 'code': response.status_code}
                
        except requests.exceptions.Timeout:
            if attempt < retry_count - 1:
                time.sleep(1)
                continue
            return {'status': 'error', 'error_type': 'timeout'}
        except Exception as e:
            return {'status': 'error', 'error_type': 'exception', 'message': str(e)}
    
    return {'status': 'error', 'error_type': 'max_retries'}

In [12]:
# Initialize collection tracking
collection_start = datetime.now()
air_quality_data = []
error_log = []

print(f"Collecting data for 500 cities:")
# log
log_step('API Collection Start', f'Beginning collection for {len(top_500_cities)} cities')

# all 500
for idx, row in top_500_cities.iterrows():
    city_name = row['city']
    country = row['country']
    lat = row['lat']
    lon = row['lng']
    population = row['population']
    
    if (idx + 1) % 50 == 0:
        print(f"{idx + 1}/{len(top_500_cities)}")
    result = get_current_air_quality(lat, lon, API_KEY)
    
    if result['status'] == 'success':
        api_data = result['data']
        
        aqi = None
        category = None
        dominant_pollutant = None
        
        if 'indexes' in api_data and len(api_data['indexes']) > 0:
            index_data = api_data['indexes'][0]
            aqi = index_data.get('aqi', None)
            category = index_data.get('category', None)
            dominant_pollutant = index_data.get('dominantPollutant', None)
        
        air_quality_data.append({
            'city': city_name,
            'country': country,
            'lat': lat,
            'lon': lon,
            'aqi': aqi,
            'aqi_category': category,
            'dominant_pollutant': dominant_pollutant,
            'collection_timestamp': datetime.now().isoformat(),
            'status': 'success'
        })
    else:
        # error log
        error_log.append({
            'city': city_name,
            'country': country,
            'error_type': result.get('error_type', 'unknown'),
            'timestamp': datetime.now().isoformat()
        })
        
        air_quality_data.append({
            'city': city_name,
            'country': country,
            'lat': lat,
            'lon': lon,
            'aqi': None,
            'aqi_category': None,
            'dominant_pollutant': None,
            'collection_timestamp': datetime.now().isoformat(),
            'status': 'error'
        })
    
    # rate limit
    time.sleep(0.25)

collection_end = datetime.now()
duration = (collection_end - collection_start).total_seconds()

print(f"Done! {duration} seconds)")

# log
log_step('API Collection Complete', f'Collected {len(air_quality_data)} records in {duration:.1f}s, {len(error_log)} errors')

[2025-11-20T14:38:30.595584] API Collection Start: Beginning collection for 500 cities
50/500
100/500
150/500
200/500
250/500
300/500
350/500
400/500
450/500
500/500
Done! 418.741052 seconds)
[2025-11-20T14:45:29.336727] API Collection Complete: Collected 500 records in 418.7s, 54 errors


In [14]:
# raw data
raw_aq_df = pd.DataFrame(air_quality_data)
raw_filename = f"data/raw_air_quality_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
raw_aq_df.to_csv(raw_filename, index=False)

# log
log_step('Data Storage', f'Saved raw API data: {raw_filename}')

print(len(error_log))
# Save error log if there are errors
if len(error_log) > 0:
    error_df = pd.DataFrame(error_log)
    error_filename = f"data/collection_errors_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    error_df.to_csv(error_filename, index=False)
    print(f"Saved errors: {error_filename}")
    # log
    log_step('Error Logging', f'Saved {len(error_log)} errors to {error_filename}')

[2025-11-20T14:45:59.477932] Data Storage: Saved raw API data: data/raw_air_quality_20251120_144559.csv
54
Saved errors: data/collection_errors_20251120_144559.csv
[2025-11-20T14:45:59.480788] Error Logging: Saved 54 errors to data/collection_errors_20251120_144559.csv


## Part 3: Data Cleaning and Standardization

Standardize field names, formats, and units across datasets.

In [15]:
# Load raw data for cleaning
cities_clean = top_500_cities.copy()
aq_clean = raw_aq_df.copy()

# log
log_step('Cleaning Start', 'Beginning data cleaning and standardization')

[2025-11-20T14:46:38.559416] Cleaning Start: Beginning data cleaning and standardization


In [16]:
# Standardize column names
cities_clean = cities_clean.rename(columns={
    'lat': 'latitude',
    'lng': 'longitude'
})

aq_clean = aq_clean.rename(columns={
    'lat': 'latitude',
    'lon': 'longitude'
})

# log
log_step('Cleaning - Column Names', 'Standardized column names to snake_case')

[2025-11-20T14:47:04.684477] Cleaning - Column Names: Standardized column names to snake_case


In [17]:
# Standardize data types
cities_clean['population'] = pd.to_numeric(cities_clean['population'], errors='coerce')
cities_clean['latitude'] = pd.to_numeric(cities_clean['latitude'], errors='coerce')
cities_clean['longitude'] = pd.to_numeric(cities_clean['longitude'], errors='coerce')

aq_clean['aqi'] = pd.to_numeric(aq_clean['aqi'], errors='coerce')
aq_clean['latitude'] = pd.to_numeric(aq_clean['latitude'], errors='coerce')
aq_clean['longitude'] = pd.to_numeric(aq_clean['longitude'], errors='coerce')

cities_clean['city'] = cities_clean['city'].str.strip()
cities_clean['country'] = cities_clean['country'].str.strip()

aq_clean['city'] = aq_clean['city'].str.strip()
aq_clean['country'] = aq_clean['country'].str.strip()

# log
log_step('Cleaning - Data Types', 'Standardized numeric and string data types')

[2025-11-20T14:47:06.899844] Cleaning - Data Types: Standardized numeric and string data types


In [18]:
# Handle missing values
missing_aqi = aq_clean['aqi'].isna().sum()
print(f"Missing AQI: {missing_aqi} ({missing_aqi/len(aq_clean)*100:.1f}%)")

# Flag data quality
aq_clean['data_quality_flag'] = aq_clean['aqi'].notna().map({True: 'complete', False: 'missing_aqi'})
print(aq_clean['data_quality_flag'].value_counts())

# log
log_step('Cleaning - Missing Values', f'{missing_aqi} cities missing AQI data, flagged for transparency')

Missing AQI: 54 (10.8%)
data_quality_flag
complete       446
missing_aqi     54
Name: count, dtype: int64
[2025-11-20T14:59:08.994920] Cleaning - Missing Values: 54 cities missing AQI data, flagged for transparency


In [19]:
# Check categorical values
print(f"AQI categories: {aq_clean['aqi_category'].unique()}")
print(f"Pollutants: {aq_clean['dominant_pollutant'].unique()}")

log_step('Cleaning - Categories', 'Reviewed and standardized categorical values')

AQI categories: ['Good air quality' 'Poor air quality' 'Excellent air quality'
 'Low air quality' 'Moderate air quality' None]
Pollutants: ['pm25' 'pm10' 'o3' None 'no2']
[2025-11-20T14:59:20.358595] Cleaning - Categories: Reviewed and standardized categorical values


## Part 4: Data Integration

Merge population and air quality data into a single integrated table.

In [20]:
# log
log_step('Integration Start', 'Beginning merge of population and air quality data')

# Select relevant columns from each dataset
cities_cols = ['city', 'country', 'latitude', 'longitude', 'population', 'iso2', 'iso3']
aq_cols = ['city', 'country', 'aqi', 'aqi_category', 'dominant_pollutant', 
           'collection_timestamp', 'data_quality_flag']

cities_for_merge = cities_clean[cities_cols]
aq_for_merge = aq_clean[aq_cols]

[2025-11-20T14:59:41.440665] Integration Start: Beginning merge of population and air quality data


In [21]:
# Merge datasets
integrated_data = cities_for_merge.merge(
    aq_for_merge,
    on=['city', 'country'],
    how='left',
    indicator=True
)

print(integrated_data['_merge'].value_counts())
print(f"Shape: {integrated_data.shape}")

# log
log_step('Integration - Merge', f'Merged {len(integrated_data)} records')

_merge
both          508
left_only       0
right_only      0
Name: count, dtype: int64
Shape: (508, 13)
[2025-11-20T14:59:53.117175] Integration - Merge: Merged 508 records


In [22]:
# Drop merge indicator column
integrated_data = integrated_data.drop('_merge', axis=1)

print(integrated_data.head())

        city    country  latitude  longitude  population iso2 iso3   aqi  \
0      Tokyo      Japan   35.6870   139.7495  37785000.0   JP  JPN  64.0   
1    Jakarta  Indonesia   -6.1750   106.8275  33756000.0   ID  IDN  63.0   
2      Delhi      India   28.6100    77.2300  32226000.0   IN  IND  14.0   
3  Guangzhou      China   23.1300   113.2600  26940000.0   CN  CHN  60.0   
4     Mumbai      India   19.0761    72.8775  24973000.0   IN  IND  19.0   

       aqi_category dominant_pollutant        collection_timestamp  \
0  Good air quality               pm25  2025-11-20T14:38:31.279676   
1  Good air quality               pm10  2025-11-20T14:38:32.165895   
2  Poor air quality               pm10  2025-11-20T14:38:32.973334   
3  Good air quality               pm10  2025-11-20T14:38:33.801853   
4  Poor air quality               pm10  2025-11-20T14:38:34.638730   

  data_quality_flag  
0          complete  
1          complete  
2          complete  
3          complete  
4          c

## Part 5: Validation Checks

Run validation checks for duplicates, missing values, and data consistency.

In [23]:
# log
log_step('Validation Start', 'Running final validation checks on integrated data')

[2025-11-20T15:00:44.940320] Validation Start: Running final validation checks on integrated data


In [24]:
# Check for duplicates
duplicates = integrated_data.duplicated(subset=['city', 'country'], keep=False)
num_dups = duplicates.sum()

print(f"Duplicates: {num_dups}")

if num_dups > 0:
    print(integrated_data[duplicates][['city', 'country', 'population', 'aqi']])
    # Remove duplicates, keeping first occurrence
    integrated_data = integrated_data.drop_duplicates(subset=['city', 'country'], keep='first')
    print(f"Removed {num_dups} duplicates")
    # log
    log_step('Validation - Duplicates', f'Removed {num_dups} duplicate records')
else:
    # log
    log_step('Validation - Duplicates', 'No duplicates found')

Duplicates: 16
         city country  population   aqi
116    Suzhou   China   5324476.0  21.0
117    Suzhou   China   5324476.0  36.0
150   Taizhou   China   4512762.0  35.0
151   Taizhou   China   4512762.0  38.0
163    Suzhou   China   4330000.0  21.0
164    Suzhou   China   4330000.0  36.0
208    Fuzhou   China   3671192.0  68.0
209    Fuzhou   China   3671192.0  50.0
213    Fuzhou   China   3614866.0  68.0
214    Fuzhou   China   3614866.0  50.0
219  Baicheng   China   3571505.0  80.0
220  Baicheng   China   3571505.0  77.0
391   Taizhou   China   2162461.0  35.0
392   Taizhou   China   2162461.0  38.0
505  Baicheng   China   1551378.0  80.0
506  Baicheng   China   1551378.0  77.0
Removed 16 duplicates
[2025-11-20T15:00:59.139596] Validation - Duplicates: Removed 16 duplicate records


In [25]:
# Check missing values
missing_summary = integrated_data.isnull().sum()
print(missing_summary[missing_summary > 0])

missing_dict = missing_summary[missing_summary > 0].to_dict()
# log
log_step('Validation - Missing Values', f'Missing value counts: {missing_dict}')

aqi                   54
aqi_category          54
dominant_pollutant    54
dtype: int64
[2025-11-20T15:01:13.310244] Validation - Missing Values: Missing value counts: {'aqi': 54, 'aqi_category': 54, 'dominant_pollutant': 54}


In [26]:
# Check data ranges
print(f"Population: {integrated_data['population'].min():,.0f} to {integrated_data['population'].max():,.0f}")
print(f"AQI: {integrated_data['aqi'].min():.1f} to {integrated_data['aqi'].max():.1f}")
print(f"Lat: {integrated_data['latitude'].min():.2f} to {integrated_data['latitude'].max():.2f}")
print(f"Lon: {integrated_data['longitude'].min():.2f} to {integrated_data['longitude'].max():.2f}")

# Check for invalid coordinates
invalid_coords = integrated_data[
    (integrated_data['latitude'] < -90) | 
    (integrated_data['latitude'] > 90) |
    (integrated_data['longitude'] < -180) | 
    (integrated_data['longitude'] > 180)
]
print(f"Invalid coords: {len(invalid_coords)}")

# log
log_step('Validation - Data Ranges', 'All data ranges validated')

Population: 1,543,000 to 37,785,000
AQI: 0.0 to 98.0
Lat: -37.81 to 59.94
Lon: -123.10 to 153.03
Invalid coords: 0
[2025-11-20T15:01:28.806860] Validation - Data Ranges: All data ranges validated


In [27]:
# completeness
completeness = (1 - integrated_data.isnull().sum() / len(integrated_data)) * 100
print("Completeness:")
for col in integrated_data.columns:
    print(f"  {col}: {completeness[col]:.1f}%")

# log
log_step('Validation - Completeness', f'Overall data completeness calculated')

Completeness:
  city: 100.0%
  country: 100.0%
  latitude: 100.0%
  longitude: 100.0%
  population: 100.0%
  iso2: 100.0%
  iso3: 100.0%
  aqi: 89.1%
  aqi_category: 89.1%
  dominant_pollutant: 89.1%
  collection_timestamp: 100.0%
  data_quality_flag: 100.0%
[2025-11-20T15:01:44.359612] Validation - Completeness: Overall data completeness calculated


## Part 6: Final Dataset Creation

Create final clean, integrated CSV file.

In [28]:
# reorder
column_order = [
    'city', 'country', 'iso2', 'iso3',
    'latitude', 'longitude',
    'population',
    'aqi', 'aqi_category', 'dominant_pollutant',
    'data_quality_flag', 'collection_timestamp'
]

final_data = integrated_data[column_order]

print(f"Final shape: {final_data.shape}")
print(final_data.head())

Final shape: (496, 12)
        city    country iso2 iso3  latitude  longitude  population   aqi  \
0      Tokyo      Japan   JP  JPN   35.6870   139.7495  37785000.0  64.0   
1    Jakarta  Indonesia   ID  IDN   -6.1750   106.8275  33756000.0  63.0   
2      Delhi      India   IN  IND   28.6100    77.2300  32226000.0  14.0   
3  Guangzhou      China   CN  CHN   23.1300   113.2600  26940000.0  60.0   
4     Mumbai      India   IN  IND   19.0761    72.8775  24973000.0  19.0   

       aqi_category dominant_pollutant data_quality_flag  \
0  Good air quality               pm25          complete   
1  Good air quality               pm10          complete   
2  Poor air quality               pm10          complete   
3  Good air quality               pm10          complete   
4  Poor air quality               pm10          complete   

         collection_timestamp  
0  2025-11-20T14:38:31.279676  
1  2025-11-20T14:38:32.165895  
2  2025-11-20T14:38:32.973334  
3  2025-11-20T14:38:33.801853  

In [29]:
# Save final integrated dataset
final_filename = 'data/integrated_cities_air_quality_final.csv'
final_data.to_csv(final_filename, index=False)

print(f"Saved: {final_filename}")
# log
log_step('Final Dataset', f'Created final integrated dataset: {final_filename}')

Saved: data/integrated_cities_air_quality_final.csv
[2025-11-20T15:03:47.068251] Final Dataset: Created final integrated dataset: data/integrated_cities_air_quality_final.csv


## Part 7: Curation Log Export

Export all curation steps for reproducibility.

In [30]:
# Save curation log
log_df = pd.DataFrame(curation_log)
log_filename = f"data/curation_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
log_df.to_csv(log_filename, index=False)

print(f"Log saved: {log_filename} ({len(curation_log)} steps)")
print(log_df.head(10))

Log saved: data/curation_log_20251120_150359.csv (25 steps)
                    timestamp                          step  \
0  2025-11-20T14:26:27.387966                     Data Load   
1  2025-11-20T14:26:46.632955   Validation - Missing Values   
2  2025-11-20T14:28:21.702501       Validation - Duplicates   
3  2025-11-20T14:28:32.216926  Validation - Population Data   
4  2025-11-20T14:28:54.540540      Validation - Coordinates   
5  2025-11-20T14:29:13.695526                Data Selection   
6  2025-11-20T14:33:56.297092          API Collection Start   
7  2025-11-20T14:38:30.595584          API Collection Start   
8  2025-11-20T14:45:29.336727       API Collection Complete   
9  2025-11-20T14:45:43.119919                  Data Storage   

                                             details  
0        Loaded 48059 cities from SimpleMaps dataset  
1                      5 columns have missing values  
2     Found 3694 duplicate city-country combinations  
3               99.5% of c

## Week 3/4 Complete

Successfully completed:
- ✓ SimpleMaps dataset validation
- ✓ Large-scale API data collection (500 cities)
- ✓ Error handling and rate limiting
- ✓ Raw data storage with timestamps
- ✓ Data cleaning and standardization
- ✓ Dataset integration and merging
- ✓ Validation checks (duplicates, missing values, ranges)
- ✓ Final CSV creation
- ✓ Curation log documentation